In [1]:
import os

from PIL import Image
from points import NumberPoints
import random

In [2]:
random.seed(1714)

# constants
boundary_width = 5
background_colour = "#808080"
yellow = "#fffe04"
blue = "#0003f9"
point_sep = 20
min_point_radius = 8
max_point_radius = 16
init_size = 512
mode = "RGB"

IMG_DIR = "../images/output_pngs"
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, "yellow"), exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, "blue"), exist_ok=True)

In [3]:
# halberda point num and ratios
min_point_num = 4
max_point_num = 16

ratios = [2 / 5, 1 / 2, 2 / 3, 3 / 4, 4 / 5, 5 / 6, 7 / 8, 8 / 9, 9 / 10, 10 / 11, 11 / 12]

# All possibilities

In [4]:
p = [[(b, int(b / a)) for a in ratios if max_point_num >= b / a == round(b / a)] for b in range(min_point_num, max_point_num + 1)]
pos = [item for sublist in p for item in sublist]

In [5]:
def create_image(n1, n2, name_, equalized):
    img = Image.new(mode, (init_size, init_size), color=background_colour)
    number_points = NumberPoints(img, init_size, yellow=yellow, blue=blue)
    point_array = number_points.design_n_points(n1, "yellow")
    point_array = number_points.design_n_points(n2, "blue", point_array=point_array)
    if equalized:
        point_array = number_points.equalize_areas(point_array)

    img = number_points.draw_points(point_array)

    if n1 > n2:
        img.save(os.path.join(IMG_DIR, "yellow", name_))
    else:
        img.save(os.path.join(IMG_DIR, "blue", name_))

In [6]:
def create_and_save(n1, n2, equalized, tag=""):
    name = f"img_{n1}_{n2}_{tag}{'_equalized' if equalized else ''}.png"
    create_image(n1, n2, name, equalized)

for i in range(100):
    for pair in pos:
        create_and_save(pair[0], pair[1], equalized=True, tag=i)
        create_and_save(pair[1], pair[0], equalized=True, tag=i)
        create_and_save(pair[0], pair[1], equalized=False, tag=i)
        create_and_save(pair[1], pair[0], equalized=False, tag=i)

In [13]:
pos

[(4, 10),
 (4, 8),
 (4, 6),
 (4, 5),
 (5, 10),
 (5, 6),
 (6, 15),
 (6, 12),
 (6, 9),
 (6, 8),
 (7, 14),
 (7, 8),
 (8, 16),
 (8, 12),
 (8, 10),
 (8, 9),
 (9, 12),
 (9, 10),
 (10, 15),
 (10, 12),
 (10, 11),
 (11, 12),
 (12, 16),
 (12, 15),
 (14, 16)]